In [2]:
# import libraries
import subprocess
import json
import os
from dotenv import load_dotenv
from constants import *
from web3 import Web3
from eth_account import Account
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy

load_dotenv()

True

In [3]:
# import mnemonic from env
mnemonic = os.getenv("mnemonic")

In [4]:
# connect to local ETH/ geth
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [6]:
# create a function called `derive_wallets`
def derive_wallets(mnemonic, coin, numderive):
    command = f'php derive -g --mnemonic="{mnemonic}" --coin="{coin}" --numderive="{numderive}" --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [7]:
# create a dictionary object called coins to store the output from `derive_wallets`.
coins = {"eth", "btc-test", "btc"}
numderive = 3

In [8]:
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(mnemonic, coin, numderive=3)

In [9]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "17L89rx13GvEaAyrQjEJ4JnJkzbFYnsaE2",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "KxvTiR5stQ9zpS542bhSjFF6wJ6dsuHeuPEuo5iFJ45QBQ9RnN7E",
            "pubkey": "039023ccc11ea158bfa3b434d6cc80c5c235c88c05919bf51aa5edb952d781f79f",
            "pubkeyhash": "456ea0fa12c0dddc44b87891ba98b077433a7785",
            "xprv": "xprvA3UDoR5D2Pyc4kHDc76rVn2AeVNgCAcZFyq7z88GS6szZJaviLpSZGfdbC3NgFf8gpQC4AZkd5DDLPc7DxS7vT797tVYzsG42bBJtybvZdc",
            "xpub": "xpub6GTaCvc6rmXuHEMgi8drruxuCXDAbdLQdCkinWXszSQyS6v5Ft8h74z7SVizWcFUY6fA875H7o37FM1unXjn4KAPtduD3jk7uXru2Xat3Q9"
        },
        {
            "address": "1JHbxvtJsjARiR9EKsikSstZewkWJWTZwk",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "KxgACaPxVynmLsdWGikZAzUTVT85YCcX9rbw4sD274MALskJhNNK",
            "pubkey": "02e0e88941e5c49de7ea6ddef78a68bb16a7d04a5e6c7fb35b1ac67fe0acc4512b",
            "pubkeyhash":

In [10]:
eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']

print(eth_PrivateKey)
print(btc_PrivateKey)

0x91931b08b93efc2befa76fac840f5948392d527b9817617fa5c1dc131f73fecd
cNDjtsoGrP3PJCUMS3mSu2Mvo73rXRBM6tD3NZsZtdCBVTKxFRCt


In [11]:
# create a function that convert the privkey string in a child key to an account object.
def priv_key_to_account(coin,priv_key):
    print(coin)
    print(priv_key)
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [12]:
def create_tx(coin, account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [13]:
# create a function to hold Ethereum 
eth_acc = priv_key_to_account(ETH, derive_wallets(mnemonic, ETH,5)[0]['privkey'])

eth
0x91931b08b93efc2befa76fac840f5948392d527b9817617fa5c1dc131f73fecd


In [14]:
# create a function to send txn
def send_txn(coin,account,recipient, amount):
    txn = create_tx(coin, account, recipient, amount)
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        tx_btctest = create_tx(coin, account, recipient, amount)
        signed_txn = account.sign_transaction(txn)
        print(signed_txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)

# BTC-TEST Transactions

In [15]:
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)

btc-test
cNDjtsoGrP3PJCUMS3mSu2Mvo73rXRBM6tD3NZsZtdCBVTKxFRCt


In [17]:
create_tx(BTCTEST,btc_acc,"mtwKEtSHmt9xuBSFLeWNXWQDPKTEK1rhtV", 0.001)

'{"unspents":[{"amount":87000,"confirmations":0,"script":"76a91493351a45250369b5ecd6ed98a72cb5d45540a2c488ac","txid":"4f3f57ea21db2b08f85a2d47568aefdd0da08e89979d6b3203d872b5426ed4df","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":66821,"confirmations":0,"script":"76a91493351a45250369b5ecd6ed98a72cb5d45540a2c488ac","txid":"028801a053202d596f4b914841ea4d2f7875ff05c3d22d5ed42eff526fec3edb","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":50000,"confirmations":0,"script":"76a91493351a45250369b5ecd6ed98a72cb5d45540a2c488ac","txid":"7cc277fab7f3d82bcd37c795d0d73c2c0d3532ad4a495abbb5d08997d0e6bd5b","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mtwKEtSHmt9xuBSFLeWNXWQDPKTEK1rhtV",100000],["mtwKEtSHmt9xuBSFLeWNXWQDPKTEK1rhtV",50577]]}'

In [18]:
# send BTC transaction
send_txn(BTCTEST,btc_acc,"mg5XJJKR4jgf5PTMkRDBJZzn6hfyVfyHzS", 0.001)

0100000003dfd46e42b572d803326b9d97898ea00dddef8a56472d5af8082bdb21ea573f4f010000006b48304502210093b866007a2638cded85c91704e6a8d874d82eacc4e891310695662692f96dbf0220510e9edd855c6133f1d9670057d72b8ff36a43148728d993c2534fcac98b97bf0121023b5a8a5a23a52b10047c53b95585a9eb742fa67342cd81e4982c70d282ae45aeffffffffdb3eec6f52ff2ed45e2dd2c305ff75782f4dea4148914b6f592d2053a0018802000000006b483045022100ab64468b20236979650b230dd3b875a31a6bee65c31e07e0e5c85c644ef31dae022064b307293b9a19ed406f8782ca3041c61d7115087c3dcc7b0cb6e8cf357221f60121023b5a8a5a23a52b10047c53b95585a9eb742fa67342cd81e4982c70d282ae45aeffffffff5bbde6d09789d0b5bb5a494aad32350d2c3cd7d095c737cd2bd8f3b7fa77c27c010000006a473044022048ce7adb59217c44dd9a28cacc1f6db9de2f2b6a0822c1f283c1aeab3015a418022063e43c0933e3c228180fb90344a0a956e32cd1362cac244ab4584b669addf8980121023b5a8a5a23a52b10047c53b95585a9eb742fa67342cd81e4982c70d282ae45aeffffffff02a0860100000000001976a9140628dc7f90e528063bf2b9685a6085488f8bba6d88ac91c50000000000001976a91493351a4525

# ETH Transactions

In [5]:
# connecting to HTTP and confirming I am connected to blockchain
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0x91931b08b93efc2befa76fac840f5948392d527b9817617fa5c1dc131f73fecd"))

In [6]:
w3.isConnected()

False

0